# Router Network


### 1.1 Dataset

The following notebook studies dataset that belongs to:

**CAIDA Skitter Router-Level Topology and Degree Distribution**: [Dataset Link](https://www.caida.org/catalog/datasets/router-adjacencies/)

CAIDA released the adjacency matrix of the Internet router-level graph computed from the ITDK0304 skitter data collected between Apr 21 and May 8, 2003. Routers act as points and their direct connections through cables form the links. Exploring this setup helps uncover how the internet's core infrastructure works.


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

In [2]:
G = nx.Graph()

with open("static/data.txt", "r") as file:
    f1 = csv.reader(file, delimiter='\t')
    for row in f1:
        node1, node2 = map(int, row)
        G.add_edge(node1, node2)

total_nodes = G.number_of_nodes()
total_edges = G.number_of_edges()

print(f'Number of nodes: {total_nodes}')
print(f'Number of edges: {total_edges}')

Number of nodes: 192244
Number of edges: 609066


The dataset contains **192244** nodes and **609066** edges.

In [3]:
# plt.figure(figsize=(10, 10))
# nx.draw(G, with_labels=False, node_size=5)
# plt.title("Router Network")
# plt.show()

### 1.2 Network characterization

#### 1.2.1 Giant Component

In the context of a router network, the giant component represents a highly interconnected subset of routers within the overall network topology. This subset likely includes routers that are crucial for facilitating communication and data transfer across the network.

In [4]:
giant_component = max(nx.connected_components(G), key=len)

G_giant = G.subgraph(giant_component)


giant_component_nodes = G_giant.number_of_nodes()
giant_component_edges = G_giant.number_of_edges()

print(f'Number of nodes in the giant component: {giant_component_nodes}')
print(f'Number of edges in the giant component: {giant_component_edges}\n')


giant_nodes_percentage = 100 * giant_component_nodes / total_nodes
giant_edges_percentage = 100 * giant_component_edges / total_edges

giant_nodes_percentage_formatted = "{:.2f}".format(giant_nodes_percentage)
giant_edges_percentage_formatted = "{:.2f}".format(giant_edges_percentage)

print(f'Percentage of nodes that are present in the giant component: {giant_nodes_percentage_formatted}%')
print(f'Number of edges in the giant component: {giant_edges_percentage_formatted}%')

Number of nodes in the giant component: 190914
Number of edges in the giant component: 607610

Percentage of nodes that are present in the giant component: 99.31%
Number of edges in the giant component: 99.76%


As we can see in the cell above the giant component contains significant portion of the whole dataset. **190914 nodes**, which is more than **99.3%** of the whole dataset and **607610 edges**, which is almost 99.8% of the whole dataset.

This big interconnectedness implies robustness and resilience against node failures or disruptions. The giant component ensures alternative paths for data transmission, therefore it is extremely efficient in routing the data packets.

In [23]:
# Average node degree 
from cmath import log


average_degree = 2 * total_edges / total_nodes

# Clustering Coefficient
clustering_coefficient = nx.average_clustering(G)

# Average path length
try:
    # average_path_length = nx.average_shortest_path_length(G)
    diameter = nx.approximation.diameter(G)
except nx.NetworkXError:
    print("Graph is not connected")
    # average_path_length = nx.average_shortest_path_length(G_giant)
    diameter_approximation = nx.approximation.diameter(G_giant)



print(f'Average Degree: {average_degree:.2f}')
print(f'Clustering Coefficient: {clustering_coefficient:.2f}\n')

complexity_diameter = [giant_component_nodes * (giant_component_nodes + giant_component_edges), int((log(giant_component_nodes) * (giant_component_nodes ** 2)).real)]

print(f'Complexity of diameter algorithm reaches {complexity_diameter[0]} or {complexity_diameter[1]} operations')

print(f'Diameter approximation: {diameter_approximation} \n')



complexity_average_shortest_path = giant_component_nodes ** 3

print(f'Complexity of Average shortest Path reaches {complexity_average_shortest_path} operations')

Graph is not connected
Average Degree: 6.34
Clustering Coefficient: 0.16

Complexity of diameter algorithm reaches 152449410936 or 443194201026 operations
Diameter approximation: 26 

Complexity of Average shortest Path reaches 6958463139271944 operations


**Average node degree of 6.34** suggests a moderate level of connectivity within the network, enough to allow for routers to find alternative paths for data transmission, but not high enough to ensure smaller complexity of the network.

**Clustering Coefficient of 0.16** exhibits a relatively low level of clustering. This indicates that neighboring nodes in the network are not highly likely to be connected to each other. The main conclusion is that 




However calculating the average path length and diamater exceeds computational capacity of my computer or Google Colab. 

For diameter algorithm takes O(V⋅(V+E)) or O(V*V*log(V)) time, which corresponds to 1.52e+11 or 4.43e+11 operations.

The **approximation of diameter** of the network is **26** edges. This means that the router network is relatively well-connected, as it doesn't take an excessively large number of steps to traverse between any regions of the world. Exchanged information will be transmitted without big latency.


However, the computional complexity of it is O(V^3) and with 190914 nodes in the giant component this number is almost 7 quadrillion (6,96e+15).

However, I found a study with similar router dataset, that is ISI-99 with 228263 nodes. 

```
https://www.researchgate.net/publication/3960121_Internet_Topology_Modeler_Based_on_Map_Sampling
```

Calculated **average path length is 9.51** and the path length distribution is displayed on the image below.

![alt text](static/average_path_length.png "Title")


In [28]:
diamater = nx.diameter(G_giant)

print(f'Diameter: {diameter}')